# ![.right](https://lh6.ggpht.com/wUrTIhpHPVqw_VPCdhbSiF5TXlBlLyRHdH1gsII_y5NkgYKzbbj7cC2l6AGoVq-JN0U=w100 "MSDS 7331 Data Mining - Project 1")


# Project 3: BEER ME! 

### a recommendation system for those who are thirsty but don't know it yet

---



# Team Members

- Chris Woodard
- Claire Chu
- Nathan Mowat
- Bill Kerneckel

---


# Rubic

- [Business Understanding](#dp1) 

- [Data Understanding 1](#dp2)

- [Data Understanding 2](#me1)

- [Modeling and Evaluation 1](#me2)

- [Modeling and Evaluation 2](#me3)

- [Modeling and Evaluation 3](#me4)

- [Modeling and Evaluation 4](#me5)

- [Deployment](#d)

- [Exceptional Work](#ew)

- [Appendix](#a)

<hr>



# OBJECTIVE: 
We found a comprehensive list of beer reviews from beeradvocate and ratebeer. We want to use these reviews to generate a recommendation system that will suggest potential beers that a user might like based on their previous beer ratings and reviews:
- Establish a solid distance metric to determine the similiarity amongst different users
- Utilize this metric to recommend other beers that a user might like

We hope that the collaborative filtering on a custom recommendation system completed in this notebook will help make beer drinkers thirsty for beers they have not tried yet.

source: https://data.world/socialmediadata/beeradvocate
source: http://nbviewer.jupyter.org/gist/glamp/20a18d52c539b87de2af
source: https://snap.stanford.edu/data/web-BeerAdvocate.html
source: https://github.com/yhat/Beer-Rec-Flask/blob/master/README.md

# Business Understanding

---

<span style="color:red">10 Points - Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?</span>

This data was collected from Beeradvocate.com and ratebeer.com. Both websites are used by beer drinkers to not only stay informed about the latest in beer news, but to also track and review the beers each user has tried. As an avid beer drinker myself, these websites are useful to me in that they help me avoid re-ordering beers i've previously tried and did not like. This helps me save money and stomach space when i'm ordering drinks at the bar. 

The dataset we are analyzing has a total of 13 columns including: 
-brewery Id
-brewery name
-review time
-overall review rating
-review of aroma
-review of appears
-username of reviewer
-beer style
-review palate
-review taste
-beer name
-alcohol percentage by volume
-beer ID

A good algorithm will be able to suggest a beer that the user will drink and rate at least a 4 or above in their overall rating (we chose 4 because this is the median review rating). If a user reviews the suggested beer at less than 4, our recommender system is not effective. 

## need help here: Statistically, we will use a ....

# Data Understanding

---

<span style="color:red"> 10 Points - Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?</span>

Insert content here!.......

In [1]:
#load python libaries
import numpy as np
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as pl
import seaborn as sns
import glob
import sys
import os
import warnings
import ggplot
import datetime
import time
from datetime import date

In [ ]:
%matplotlib inline

In [15]:
#load data file
beerDF = pd.read_csv('/Users/macnificent/Desktop/beer_reviews.csv') 

#you'll have to change the file path to your working directory


In [14]:
#combining dataframes 
cdf = data_file.copy()
cdf.type()


AttributeError: 'DataFrame' object has no attribute 'type'

In [16]:
# dropping columns we are not using for the analysis
beerDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [17]:
beerDF.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [22]:
summary = beerDF.describe()
summary = summary.transpose()
summary

,count,mean,std,min,25%,50%,75%,max
brewery_id,1586614.0,3.130099e+03,5.578104e+03,1.000000e+00,1.430000e+02,4.290000e+02,2.372000e+03,2.800300e+04
review_time,1586614.0,1.224089e+09,7.654427e+07,8.406720e+08,1.173224e+09,1.239203e+09,1.288568e+09,1.326285e+09
review_overall,1586614.0,3.815581e+00,7.206219e-01,0.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
review_aroma,1586614.0,3.735636e+00,6.976167e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_appearance,1586614.0,3.841642e+00,6.160928e-01,0.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_palate,1586614.0,3.743701e+00,6.822184e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_taste,1586614.0,3.792860e+00,7.319696e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
beer_abv,1518829.0,7.042387e+00,2.322526e+00,1.000000e-02,5.200000e+00,6.500000e+00,8.500000e+00,5.770000e+01
beer_beerid,1586614.0,2.171279e+04,2.181834e+04,3.000000e+00,1.717000e+03,1.390600e+04,3.944100e+04,7.731700e+04


In [30]:
#what is the most popular beer?
beerDF.beer_name.describe()

count           1586614
unique            56857
top       90 Minute IPA
freq               3290
Name: beer_name, dtype: object

In [36]:
#what are the top 10 most popular beers?
from collections import Counter
topbeer = Counter(beerDF.beer_name)
topbeer.most_common(10)

[('90 Minute IPA', 3290),
 ('India Pale Ale', 3130),
 ('Old Rasputin Russian Imperial Stout', 3111),
 ('Sierra Nevada Celebration Ale', 3000),
 ('Two Hearted Ale', 2728),
 ('Stone Ruination IPA', 2704),
 ('Arrogant Bastard Ale', 2704),
 ('Sierra Nevada Pale Ale', 2587),
 ('Stone IPA (India Pale Ale)', 2575),
 ('Pliny The Elder', 2527)]

In [51]:
#use the top three most reviewed beers to see if there are any reviewers that have written reviews on all three
beer_1, beer_2, beer_3 = "90 Minute IPA", "India Pale Ale", "Old Rasputin Russian Imperial Stout"

beer_1_reviewers = beerDF[beerDF.beer_name==beer_1].review_profilename.unique()
beer_2_reviewers = beerDF[beerDF.beer_name==beer_2].review_profilename.unique()
beer_3_reviewers = beerDF[beerDF.beer_name==beer_3].review_profilename.unique()
common_reviewers = set(beer_1_reviewers) & set(beer_2_reviewers) & set(beer_3_reviewers)
print ("Users in the sameset: %d " % len(common_reviewers))
list(common_reviewers)[:20]

###having an issue with this because the output isnt the intersection of list 1, 2, 3??

Users in the sameset: 809 


['ngeunit1',
 'jsh420',
 'Dantes',
 'mobyfann',
 'sprucetip',
 'TikkiGodxxx',
 'stcules',
 'Richardberg',
 'dyan',
 'Jeffo',
 'jmz62565',
 'Huhzubendah',
 'SupaCelt',
 'Gehrig',
 'PhantomVodoo',
 'GeoffRizzo',
 'zeff80',
 'Bumpon10s',
 'Phatz',
 'Marcus6679']

In [41]:
#which reviewer has the most reviews?

toprevs = Counter(beerDF.review_profilename)
toprevs.most_common(10)

[('northyorksammy', 5817),
 ('BuckeyeNation', 4661),
 ('mikesgroove', 4617),
 ('Thorpe429', 3518),
 ('womencantsail', 3497),
 ('NeroFiddled', 3488),
 ('ChainGangGuy', 3471),
 ('brentk56', 3357),
 ('Phyl21ca', 3179),
 ('WesWes', 3168)]

AttributeError: __exit__

In [122]:
cdf['salary_range'] = pd.cut(cdf['salary'],[0,50000,100000,135000,150000],4,labels=['Entry_Level','Jr_Level','Mid_Level', 'Mgmt_Level'])
print(cdf['salary_range'].head())
cdf.dropna(inplace=True)

1     Jr_Level
2    Mid_Level
3     Jr_Level
4    Mid_Level
5     Jr_Level
Name: salary_range, dtype: category
Categories (4, object): [Entry_Level < Jr_Level < Mid_Level < Mgmt_Level]


In [123]:
cdf.salary.max() #Is the salary range cut-off still valid with the new data? Yes

149993.0

In [124]:
# convert cdf variable back to "df" nomenclature
df = pd.DataFrame(cdf)

In [125]:
#manipulate employee start date 'eoddte' to get 'service' variable
#convert 'eoddte' to series
df.eod = pd.Series(df['eoddte'])
#convert 'eoddte' series to 'eoddte' datetime
df['eoddte'] = pd.to_datetime(df.eod) 
#convert eod to be just the year
df['eodyr'] = df['eoddte'].map(lambda x: x.year)

In [126]:
#convert high school graduation year "hiyr" to be numeric
df['hiyr'] = df['hiyr'].apply(pd.to_numeric, errors='coerce')
#after looking at a crosstab, we can see that there are a lot of "high school graduation year: 0"
#this is a data entry error, let's remove these entries...
df = df[df.hiyr != 0]
pd.crosstab(index=df["hiyr"],columns="hiyr")
#now the 0 entries have been removed

#subtract employee hire date from high school graduation date to get exprience
df['experience'] = df['eodyr'] - df['hiyr']

In [127]:
#get the 'service' variable by subtracting current date from hire date
today = date(2016, 12, 31)

df['service'] = today - df['eoddte']
#generate retirement potential by subtracting retirement eligibility date from todays date  
#convert retoptdte to series for conversion to datetime format
df.retoptdte = pd.Series(df['retoptdte'])
#convert 'retoptdte' series to 'eoddte' datetime
df['retoptdte'] = pd.to_datetime(df.retoptdte) 
#generate 'retpot' variable
df['retpot'] = today - df['retoptdte']

In [128]:
df_grouped = df.groupby(by=['grade','sex'])

In [129]:
df_grouped = df.groupby(by=['grade','sex'])
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.median()))
df_imputed[['grade','sex']] = df[['grade','sex']]

In [130]:
df['Hist_yr'] = df['Hist_yr'].astype(np.float64)
df_yrgroup = df.groupby(by=['Hist_yr'])

In [131]:
# creating new dataframe. Keeping original seperate.
DF_NoT = df.copy()
# perform one-hot encoding of the categorical data "salary_range" and "rno" (ethnic background).
tmp_df = pd.get_dummies(df.salary_range,prefix='salary_range')
tmp_df2 = pd.get_dummies(df.rno,prefix='rno')

DF_NoT = pd.concat((df,tmp_df),axis=1) # add back into the dataframe
DF_NoT = pd.concat((DF_NoT,tmp_df2),axis=1)

In [132]:
# Delete time/dates - can't be used in L-Regression
DF_NoT = DF_NoT.drop(['eoddte','retoptdte','salary_range', 'currgrddte', 'frscdte', 'fscdte', 'hisch', 'leqdte',
                     'loccde','ncc', 'opmtitle', 'occode', 'orga', 'orgabr', 'orgadir', 'orgadiv',  'orgasec', 
                     'postenure', 'tenure', 'supind', 'suplev', 'rno', 'rno_000010', 'rno_100100', 'rno_100001', 
                     'rno_100010', 'rno_100101', 'rno_100000', 'rno_011000', 'rno_001101', 'rno_010001', 'rno_010101',
                     'rno_001100', 'rno_010000', 'rno_010100', 'rno_001001', 'rno_001000', 'rno_000101', 'rno_000100',
                     'rno_000011', 'rno_101000', 'rno_101101', 'rno_110000', 'rno_110001', 'rno_110101', 'rno_111111',
                     'rno_10', 'rno_11', 'rno_100', 'rno_101', 'rno_1000', 'rno_1001', 'rno_1100', 'rno_1101', 'rno_10000',
                     'rno_10001', 'rno_10100', 'rno_10101', 'rno_101100', 'rno_000000', 'rno_000110', 'rno_000111', 
                     'rno_001010', 'rno_001011', 'rno_010111', 'rno_011001', 'rno_011100', 'rno_011101', 'rno_100011', 
                      'rno_101001', 'rno_1'], axis=1)

In [133]:
today = date(2016, 12, 31) #modify if more datasets are added
DF_NoT.service = DF_NoT.service / np.timedelta64(1, 'D')
DF_NoT.retpot = DF_NoT.retpot / np.timedelta64(1, 'D')



In [134]:
DF_NoT.lastpromodte = pd.Series(DF_NoT['lastpromodte'])
#convert 'eoddte' series to 'eoddte' datetime
DF_NoT['lastpromodte'] = pd.to_datetime(DF_NoT.lastpromodte) 
DF_NoT['lastpromodte'] = today - DF_NoT['lastpromodte']

#removing the day to calculate a number
DF_NoT.lastpromodte = DF_NoT.lastpromodte / np.timedelta64(1, 'D')

DF_NoT.nextwigdte = pd.Series(DF_NoT['nextwigdte'])
DF_NoT['nextwigdte'] = pd.to_datetime(DF_NoT.nextwigdte) 
DF_NoT['nextwigdte'] = today - DF_NoT['nextwigdte']
DF_NoT.nextwigdte = DF_NoT.nextwigdte / np.timedelta64(1, 'D')

DF_NoT.retdiscdte = pd.Series(DF_NoT['retdiscdte'])
DF_NoT['retdiscdte'] = pd.to_datetime(DF_NoT.retdiscdte) 
DF_NoT['retdiscdte'] = today - DF_NoT['retdiscdte']
DF_NoT.retdiscdte = DF_NoT.retdiscdte / np.timedelta64(1, 'D')

DF_NoT['IsMale'] = DF_NoT.sex=='M' 
DF_NoT.IsMale = DF_NoT.IsMale.astype(np.int)
del DF_NoT['sex']

In [135]:
DF_Reg2 = DF_NoT.copy()

In [136]:
del DF_Reg2['salary']
del DF_Reg2['salary_range_Entry_Level']
del DF_Reg2['salary_range_Mid_Level']
del DF_Reg2['salary_range_Mgmt_Level']

In [137]:
DF_Reg2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126967 entries, 2 to 17523
Data columns (total 20 columns):
Hist_yr                  126967 non-null float64
bdyr                     126967 non-null float64
grade                    126967 non-null object
hilev                    126967 non-null object
hiyr                     126967 non-null int64
install                  126967 non-null object
lastpromodte             126967 non-null float64
nextwigdte               126967 non-null float64
retdiscdte               126967 non-null float64
secyr                    126967 non-null object
step_emp                 126967 non-null object
time_in_grade            126967 non-null float64
age                      126967 non-null float64
eodyr                    126967 non-null int64
experience               126967 non-null int64
service                  126967 non-null float64
retpot                   126967 non-null float64
salary_range_Jr_Level    126967 non-null float64
rno_000001         

LEFT OVER FROM LAST ASSIGNMENT -  Based on the output above, you can see we have about 126,967 entries with 24 columns. 
New Variables created are:
- the age category derived from the "birth year" variable
- the salary_range category derived by dividing up the salary variable into 4 categories
- the salary_range_entry_level is a dummy variable established form the salary range category
- the salary_range_jr_level is a dummy variable established form the salary range category
- the salary_range_mid_level is a dummy variable established form the salary range category
- the salary_range_mgmt_level is a dummy variable established form the salary range category
- the eodyr variable which shows the year of employment
- the experience variable derived from the high school graduation year subtracted from the last date of the data set (12/31/2016)
- the serivce variable derived from the year of employment subtracted from 12/31/2016
- the retirement potential derived from the date of earliest reitrement subtracted from 12/31/2016
- the rno_000001 variable is derived from the RNO or diversity classification category where 000001 represents the Caucasian Identity Classification
- the IsMale variable is a dummy variable established from the gender classification variable.

Let's use the variables to predict future records in the NASA human resources data set
- which model can we use to predict a "junior salary range level" classification, given the variables we have
- which model can we use to predict a "caucasian" race classification, given the variables we have


# Data Understanding 2

----


<span style="color:red">10 Points - Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.</span>

Insert Content Here!

# Modeling and Evaluation 1
---

<span style="color:red">10 Points - Train and adjust parameters</span>

LEFT OVER FROM LAST ASSIGNMENT - We utilized “ACCURACY” as our evaluation metric of choice because we want to classify a salary range and the diversity (race/national origin) category.  A low false positive score is ideal because we do not want to incorrectly classify the data points.  All things being equal, it would be better if we had more false negatives. For this dataset, logic steers us to be accurate overall since we have four classes of salary range and multiple classes for the diversity classification.  An argument can be made for looking at false negatives on a per class basis. For instance, after reviewing our initial data exploration, we found that the salary range class was unevenly distributed, so we might observe a low count of false positives. This makes sense since a person in a salary range with relatively few measured data points may trigger a low amount of false negatives under some models. 

# Modeling and Evaluation 2
---

<span style="color:red">10 Points - Evaluate and Compare</span>

Insert Content Here....

# Modeling and Evaluation 3
---

<span style="color:red">10 Points - Summarize the Ramifications</span>

Insert Content Here!

# Modeling and Evaluation 4
---

<span style="color:red">20 Points - Summarize the Ramifications. </span>

Insert Content Here!

# Deployment
---


<span style="color:red">10 Points - Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? </span>

LEFT OVER FROM LAST ASSIGNMENT - All Federal Agencies are required to use and store the data elements defined by Office of Personnel Management (OPM). The data elements are defined in the following OPM website: https://www.opm.gov/policy-data-oversight/pay-leave/salaries-wages/salary-tables/datadictionary.aspx

We strongly recommend all Human Resources and Workforce Planning offices of the United States Government, especially the Office of Personnel Management (OPM) will benefit from the use of the models and analysis contained in the document. 

# Exceptional Work
---

<span style="color:red">10 Points - You have free reign to provide additional analyses or combine analyses.</span>

# Appendix